#### 라이브러리 임포트

In [4]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import os
import glob

#### 코랩 구글드라이브 마운트

In [2]:
from google.colab import drive
drive.mount('/content/drive')
# 공유폴더 -> 정리 -> 바로가기 추가로 공유 폴더 MY DRIVE로 연결

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### XML데이터를 데이터프레임으로 재구조화

In [ ]:
def make_DB(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    # 데이터 추출
    data = {
        'folder': root.find('folder').text if root.find('folder') is not None else None,
        'filename': root.find('filename').text if root.find('filename') is not None else None,
        'width': int(root.find('size/width').text) if root.find('size/width') is not None else None,
        'height': int(root.find('size/height').text) if root.find('size/height') is not None else None,
        'duration': root.find('header/duration').text if root.find('header/duration') is not None else None,
        'fps': int(root.find('header/fps').text) if root.find('header/fps') is not None else None,
        'frames': int(root.find('header/frames').text) if root.find('header/frames') is not None else None,
        'location': root.find('header/location').text if root.find('header/location') is not None else None,
        'season': root.find('header/season').text if root.find('header/season') is not None else None,
        'weather': root.find('header/weather').text if root.find('header/weather') is not None else None,
        'time': root.find('header/time').text if root.find('header/time') is not None else None,
        'population': int(root.find('header/population').text) if root.find('header/population') is not None else None,
        'eventname': root.find('event/eventname').text if root.find('event/eventname') is not None else None,
        'starttime': root.find('event/starttime').text if root.find('event/starttime') is not None else None,
        'action_duration': root.find('event/duration').text if root.find('event/duration') is not None else None
    }

    df1 = pd.DataFrame([data])

    characters = root.find('header/character').text.split(',') if root.find('header/character') is not None else []
    objects_data = [
        {
            'character': characters[i] if i < len(characters) else None,
            'objectname': obj.find('objectname').text if obj.find('objectname') is not None else None,
            'keyframe': int(obj.find('position/keyframe').text) if obj.find('position/keyframe') is not None else None,
            'keypoint_x': float(obj.find('position/keypoint/x').text) if obj.find('position/keypoint/x') is not None else None,
            'keypoint_y': float(obj.find('position/keypoint/y').text) if obj.find('position/keypoint/y') is not None else None,
            'actionname': action.find('actionname').text if action.find('actionname') is not None else None,
            'actionframe_start': int(frame.find('start').text) if frame.find('start') is not None else None,
            'actionframe_end': int(frame.find('end').text) if frame.find('end') is not None else None,
        }
        for i, obj in enumerate(root.findall('object'))
        for action in obj.findall('action')
        for frame in action.findall('frame')
    ]

    df2 = pd.DataFrame(objects_data)
    df2.insert(0, 'file_objectname', df1['filename'][0] +'('+ df2['objectname']+')')

    df1_repeated = pd.concat([df1] * len(df2), ignore_index=True)
    df = pd.concat([df1_repeated, df2],axis=1).reset_index(drop=True)

    return df

#### XMLTOXLSX

In [ ]:
def load_all_xml(folder_path):
    all_files = glob.glob(os.path.join(folder_path, '**', '*.xml'), recursive=True)
    return all_files

def xml_to_dataframe(folder_path):
    all_files = load_all_xml(folder_path)
    total_files = len(all_files)

    total_df = pd.DataFrame()

    for idx, file_path in enumerate(all_files):
      if idx > 0 and idx % 50 == 0:
        print(f'{idx} 완료 중')
      try:
        temp_df = make_DB(file_path)
        total_df = pd.concat([total_df,temp_df], ignore_index=True)
      except:
        print(f'{file_path}')

    return total_df

In [ ]:
folder_path = '/content/drive/MyDrive/이상행동데이터'
df = xml_to_dataframe(folder_path)

In [ ]:
df.shape

(70351, 24)

#### 데이터 저장

In [ ]:
final_df = df.drop_duplicates()
final_df = final_df.reset_index(drop=True)
final_df.to_excel(f'{folder_path}/이상행동데이터.xlsx', index=False)
final_df

NameError: name 'df' is not defined

----------------------------------

#### 결측값 처리

In [14]:
final_df = pd.read_excel('/content/drive/MyDrive/이상행동데이터/이상행동데이터_결측포함.xlsx') #파일이름 변경함
final_df.shape

(70339, 24)

In [15]:
missing_rows = final_df[final_df.isna().any(axis=1)]
missing_rows

,folder,filename,width,height,duration,fps,frames,location,season,weather,...,action_duration,file_objectname,character,objectname,keyframe,keypoint_x,keypoint_y,actionname,actionframe_start,actionframe_end
30026,datefight,185-6_cam02_datefight01_place02_night_spring.mp4,3840,2160,00:05:08.6,30,9258,PLACE02,SPRING,SUNNY,...,00:00:45.4,185-6_cam02_datefight01_place02_night_spring.m...,NaN,person_2,7986,2310,1204,falldown,7986,8064
30027,datefight,185-6_cam02_datefight01_place02_night_spring.mp4,3840,2160,00:05:08.6,30,9258,PLACE02,SPRING,SUNNY,...,00:00:45.4,185-6_cam02_datefight01_place02_night_spring.m...,NaN,person_2,7986,2310,1204,falldown,8321,8365
31482,datefight,194-1_cam03_datefight02_place02_night_summer.mp4,3840,2160,00:05:39.0,30,10169,PLACE02,SUMMER,SUNNY,...,00:03:09.9,194-1_cam03_datefight02_place02_night_summer.m...,NaN,person_2,7302,1713,877,touching,7302,7373
31483,datefight,194-1_cam03_datefight02_place02_night_summer.mp4,3840,2160,00:05:39.0,30,10169,PLACE02,SUMMER,SUNNY,...,00:03:09.9,194-1_cam03_datefight02_place02_night_summer.m...,NaN,person_2,7302,1713,877,touching,7457,7611
31484,datefight,194-1_cam03_datefight02_place02_night_summer.mp4,3840,2160,00:05:39.0,30,10169,PLACE02,SUMMER,SUNNY,...,00:03:09.9,194-1_cam03_datefight02_place02_night_summer.m...,NaN,person_2,7302,1713,877,touching,7994,8135
31485,datefight,194-1_cam03_datefight02_place02_night_summer.mp4,3840,2160,00:05:39.0,30,10169,PLACE02,SUMMER,SUNNY,...,00:03:09.9,194-1_cam03_datefight02_place02_night_summer.m...,NaN,person_2,7302,1713,877,touching,8346,8441
31486,datefight,194-1_cam03_datefight02_place02_night_summer.mp4,3840,2160,00:05:39.0,30,10169,PLACE02,SUMMER,SUNNY,...,00:03:09.9,194-1_cam03_datefight02_place02_night_summer.m...,NaN,person_2,7302,1713,877,touching,8823,9045
31487,datefight,194-1_cam03_datefight02_place02_night_summer.mp4,3840,2160,00:05:39.0,30,10169,PLACE02,SUMMER,SUNNY,...,00:03:09.9,194-1_cam03_datefight02_place02_night_summer.m...,NaN,person_2,7302,1713,877,touching,9133,9219
31488,datefight,194-1_cam03_datefight02_place02_night_summer.mp4,3840,2160,00:05:39.0,30,10169,PLACE02,SUMMER,SUNNY,...,00:03:09.9,194-1_cam03_datefight02_place02_night_summer.m...,NaN,person_2,7302,1713,877,touching,9462,9904
33785,robbery,177-5_cam02_robbery01_place01_day_summer.mp4,3840,2160,00:05:00.3,30,9009,PLACE01,SUMMER,SUNNY,...,00:00:48.1,177-5_cam02_robbery01_place01_day_summer.mp4(p...,NaN,person_2,7522,2720,1393,falldown,7522,7706


##### 1. character 결측 처리

In [17]:
# 표기오류로 인한 결측 처리
final_df[final_df['filename'] == '185-6_cam02_datefight01_place02_night_spring.mp4']

final_df.loc[30026,'character'] = 'F40'
final_df.loc[30027,'character'] = 'F40'

In [18]:
select_df1 = final_df[final_df['file_objectname'] == '194-1_cam03_datefight02_place02_night_summer.mp4(person_1)']
select_df2 = final_df[final_df['file_objectname'] == '194-1_cam03_datefight02_place02_night_summer.mp4(person_2)']

final_df.loc[select_df1.index, 'character'] = 'F20'
final_df.loc[select_df2.index, 'character'] = 'M20'

# 해당 데이터 population 변경
select_df = final_df[final_df['filename'] == '194-1_cam03_datefight02_place02_night_summer.mp4']
final_df.loc[select_df.index, 'population'] = 2

In [19]:
select_df = final_df[final_df['file_objectname'] == '177-5_cam02_robbery01_place01_day_summer.mp4(person_2)']
final_df.loc[select_df.index, 'character'] = 'F20'

# 해당 데이터 population 변경
select_df = final_df[final_df['filename'] == '177-5_cam02_robbery01_place01_day_summer.mp4']
final_df.loc[select_df.index, 'population'] = 2

In [20]:
select_df = final_df[final_df['file_objectname'] == '177-5_cam01_robbery01_place01_day_spring.mp4(person_2)']
final_df.loc[select_df.index, 'character'] = 'F20'

# 해당 데이터 population 변경
select_df = final_df[final_df['filename'] == '177-5_cam01_robbery01_place01_day_spring.mp4']
final_df.loc[select_df.index, 'population'] = 2

In [21]:
select_df = final_df[final_df['file_objectname'] == '178-6_cam01_robbery02_place02_night_summer.mp4(person_2)']
final_df.loc[select_df.index, 'character'] = 'M30'

# 해당 데이터 population 변경
select_df = final_df[final_df['filename'] == '178-6_cam01_robbery02_place02_night_summer.mp4']
final_df.loc[select_df.index, 'population'] = 3

In [22]:
select_df = final_df[final_df['file_objectname'] == '33-3_cam02_fight01_place02_night_spring.mp4(person_4)']
final_df.loc[select_df.index, 'character'] = 'M30'

# 해당 데이터 population 변경
select_df = final_df[final_df['filename'] == '194-1_cam03_datefight02_place02_night_summer.mp4']
final_df.loc[select_df.index, 'population'] = 4

##### 2. actionname 결측 처리

In [23]:
select_df = final_df[(final_df['filename'] == '76-5_cam02_burglary01_place01_day_summer.mp4') & (final_df['actionname'].isna())]

final_df.loc[select_df.index, 'actionname'] = 'pick up'

In [24]:
select_df = final_df[(final_df['filename'] == '66-2_cam01_burglary01_place01_night_summer.mp4') & (final_df['actionname'].isna())]

final_df.loc[select_df.index, 'actionname'] = 'pick up'

In [25]:
select_df = final_df[(final_df['filename'] == '122-2_cam01_wander02_place01_day_summer.mp4') & (final_df['actionname'].isna())]

final_df.loc[select_df.index, 'actionname'] = 'stop and go'

In [26]:
select_df = final_df[(final_df['filename'] == '122-2_cam01_wander02_place01_day_spring.mp4') & (final_df['actionname'].isna())]

final_df.loc[select_df.index, 'actionname'] = 'stop and go'

In [27]:
select_df = final_df[(final_df['filename'] == '122-6_cam01_wander02_place01_day_spring.mp4') & (final_df['actionname'].isna())]

final_df.loc[select_df.index, 'actionname'] = 'stop and go'

In [28]:
select_df = final_df[(final_df['filename'] == '85-1_cam02_vandalism01_place04_day_summer.mp4') & (final_df['actionname'].isna())]

final_df.loc[select_df.index, 'actionname'] = 'pushing'

##### 3. 결측값 확인

In [29]:
missing_rows = final_df[final_df.isna().any(axis=1)]
missing_rows

,folder,filename,width,height,duration,fps,frames,location,season,weather,...,action_duration,file_objectname,character,objectname,keyframe,keypoint_x,keypoint_y,actionname,actionframe_start,actionframe_end


-----------------------------------

#### 오류수정

In [30]:
# 행동정보 오류 수정

final_df.loc[final_df['eventname'] != final_df['folder'], 'folder'] = 'assault'

#### 데이터 저장

In [31]:
final_df.to_excel('/content/drive/MyDrive/이상행동데이터/이상행동데이터.xlsx', index=False)
final_df

,folder,filename,width,height,duration,fps,frames,location,season,weather,...,action_duration,file_objectname,character,objectname,keyframe,keypoint_x,keypoint_y,actionname,actionframe_start,actionframe_end
0,assault,10-1_cam01_assault03_place07_night_spring.mp4,3840,2160,00:05:08.2,30,9245,PLACE07,SPRING,SUNNY,...,00:02:20.2,10-1_cam01_assault03_place07_night_spring.mp4(...,F20,person_1,4382,2528,654,pushing,4382,4418
1,assault,10-1_cam01_assault03_place07_night_spring.mp4,3840,2160,00:05:08.2,30,9245,PLACE07,SPRING,SUNNY,...,00:02:20.2,10-1_cam01_assault03_place07_night_spring.mp4(...,F20,person_1,4382,2528,654,pushing,4483,4506
2,assault,10-1_cam01_assault03_place07_night_spring.mp4,3840,2160,00:05:08.2,30,9245,PLACE07,SPRING,SUNNY,...,00:02:20.2,10-1_cam01_assault03_place07_night_spring.mp4(...,F20,person_1,4382,2528,654,pushing,4546,4592
3,assault,10-1_cam01_assault03_place07_night_spring.mp4,3840,2160,00:05:08.2,30,9245,PLACE07,SPRING,SUNNY,...,00:02:20.2,10-1_cam01_assault03_place07_night_spring.mp4(...,F20,person_1,4382,2528,654,pushing,4727,4753
4,assault,10-1_cam01_assault03_place07_night_spring.mp4,3840,2160,00:05:08.2,30,9245,PLACE07,SPRING,SUNNY,...,00:02:20.2,10-1_cam01_assault03_place07_night_spring.mp4(...,F20,person_1,4382,2528,654,pushing,4792,4820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70334,swoon,118-6_cam02_swoon02_place10_day_summer.mp4,3840,2160,00:05:00.3,30,9009,PLACE10,SUMMER,SUNNY,...,00:00:04.2,118-6_cam02_swoon02_place10_day_summer.mp4(per...,F30,person_1,8883,1202,1159,falldown,8943,9009
70335,swoon,118-6_cam01_swoon02_place10_day_spring.mp4,3840,2160,00:05:00.3,30,9009,PLACE10,SPRING,SUNNY,...,00:00:12.7,118-6_cam01_swoon02_place10_day_spring.mp4(per...,F30,person_1,7976,2403,650,falldown,8602,8672
70336,swoon,118-6_cam01_swoon02_place10_day_spring.mp4,3840,2160,00:05:00.3,30,9009,PLACE10,SPRING,SUNNY,...,00:00:12.7,118-6_cam01_swoon02_place10_day_spring.mp4(per...,F30,person_1,7976,2403,650,falldown,8672,9009
70337,swoon,118-6_cam01_swoon02_place10_day_summer.mp4,3840,2160,00:05:00.3,30,9009,PLACE10,SUMMER,SUNNY,...,00:00:12.9,118-6_cam01_swoon02_place10_day_summer.mp4(per...,F30,person_1,8149,2424,743,falldown,8622,8760


#### 데이터 확인

In [ ]:
final_df['actionname'].value_counts()

,count
actionname,
punching,11141
pulling,9611
kicking,9193
pushing,9189
falldown,6417
threaten,6326
totter,5314
sitdown,4266
stop and go,2976


In [ ]:
final_df['folder'].value_counts()

,count
folder,
fight,22721
assault,16218
drunken,8948
datefight,7068
wander,3356
vandalism,2613
swoon,2582
robbery,2129
burglary,1920


In [ ]:
folder_counts = final_df.groupby('folder')['actionname'].value_counts()
fight_counts = folder_counts.loc[folder_counts.index.get_level_values(0) == 'trespass']
fight_counts

folder    actionname
trespass  pulling       338
          walking       155
          climbwall     127
          running         1
Name: count, dtype: int64

#### 데이터 분할 생성

In [9]:
for i in final_df['folder'].unique():
  print(i)

assault
drunken
kidnap
datefight
robbery
dump
trespass
burglary
wander
vandalism
fight
swoon


In [35]:
folder_path = '/content/drive/MyDrive/이상행동데이터'
for i in final_df['folder'].unique():
  select_df = final_df[final_df['folder'] == i]
  select_df.to_excel(f'{folder_path}/{i}.xlsx', index=False)